# Tuning a Random Forest Regression Model

Custom built expanding window validation functions will be used 

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

# Import your helper functions
from timeseries_model_utils import (
    expanding_window_splits,
    run_expanding_cv,
    expanding_window_grid_search
)

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error


In [6]:
df = pd.read_csv("New_csv.csv")   # replace with your file
df = df.sort_values("Year").reset_index(drop=True)
df.head()


,Year,Homicide,Assault,Sexual offences,Harm or endanger persons,"Robbery, blackmail, and extortion",Burglary,Theft,Fraud and related offences,Drug offences,...,"Public order, health, and safety offences",Traffic and vehicle offences,Offences against justice procedures and orders,Offences against government,Environmental offences,Miscellaneous offences,Total,Population,Crime_Index,Crime_Index_Lagged
0,1980,353,7208,1401,2176,426,11893,29103,19794,7839,...,28539,173948,27664,1376,3521,9651,331751,3112900,10.657297,NaN
1,1981,366,7933,1092,2299,494,12974,29654,25007,8340,...,22637,142990,17981,2027,3360,9836,294409,3124900,9.421389,10.657297
2,1982,329,7760,1298,2231,504,14387,32682,26131,9518,...,18268,104739,13957,1203,4025,7878,252119,3156100,7.988308,9.421389
3,1983,407,8741,1449,2693,657,13851,32304,28481,10346,...,16067,105649,15332,736,2252,7774,254658,3199300,7.959804,7.988308
4,1984,406,9349,1682,2651,729,13905,33492,28069,12819,...,17456,116414,16389,1300,2235,8552,274022,3227100,8.491277,7.959804


In [7]:
target_col = "Homicide"  

feature_cols = [c for c in df.columns if c not in ["Year", target_col]]

X = df[feature_cols]
y = df[target_col]


In [8]:
param_grid = {
    "n_estimators": [100, 200, 400],
    "max_depth": [3, 5, None]
}

best_params, best_score, results_df = expanding_window_grid_search(
    RandomForestRegressor,
    param_grid,
    X, y,
    initial_train_size=10,
    horizon=1,
    step=1
)

print("\nBest Parameters:", best_params)
print("Best Score:", best_score)

results_df



[GridSearch] Params {'n_estimators': 100, 'max_depth': 3} → Score = 56.6294
[GridSearch] Params {'n_estimators': 100, 'max_depth': 5} → Score = 53.8096
[GridSearch] Params {'n_estimators': 100, 'max_depth': None} → Score = 55.9151
[GridSearch] Params {'n_estimators': 200, 'max_depth': 3} → Score = 55.5886
[GridSearch] Params {'n_estimators': 200, 'max_depth': 5} → Score = 54.6186
[GridSearch] Params {'n_estimators': 200, 'max_depth': None} → Score = 56.6039
[GridSearch] Params {'n_estimators': 400, 'max_depth': 3} → Score = 54.4979
[GridSearch] Params {'n_estimators': 400, 'max_depth': 5} → Score = 54.7377
[GridSearch] Params {'n_estimators': 400, 'max_depth': None} → Score = 54.7300

Best Parameters: {'n_estimators': np.float64(100.0), 'max_depth': np.float64(5.0)}
Best Score: 53.8096394013083


,n_estimators,max_depth,score
0,100,3.0,56.629351
1,100,5.0,53.809639
2,100,NaN,55.915143
3,200,3.0,55.588644
4,200,5.0,54.618564
5,200,NaN,56.603857
6,400,3.0,54.497913
7,400,5.0,54.737694
8,400,NaN,54.730000


In [9]:

param_grid = {
    "alpha": [0.001, 0.01, 0.1, 1, 10, 50, 100],
    "max_iter": [1000, 5000, 10000]
}


In [20]:

X = X.ffill().bfill()


In [22]:
from sklearn.linear_model import Lasso

param_grid = {
    "alpha": [0.001, 0.01, 0.1, 1, 10, 50, 100],
    "max_iter": [1000, 5000, 10000],
}

best_params, best_score, results_df = expanding_window_grid_search(
    Lasso,
    param_grid,
    X,
    y,
    initial_train_size=10,
    horizon=1,
    step=1
)

print("\nBest Parameters (Lasso):", best_params)
print("Best Score:", best_score)
results_df


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.865e+01, tolerance: 9.049e+00
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.819e+01, tolerance: 9.118e+00
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.029e+02, tolerance: 9.1

[GridSearch] Params {'alpha': 0.001, 'max_iter': 1000} → Score = 83.1260


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.253e+01, tolerance: 1.810e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.026e+02, tolerance: 2.175e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.108e+02, tolerance: 2.4

[GridSearch] Params {'alpha': 0.001, 'max_iter': 5000} → Score = 85.2486


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.694e+01, tolerance: 1.810e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.012e+02, tolerance: 2.175e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.103e+02, tolerance: 2.4

...

[GridSearch] Params {'alpha': 100, 'max_iter': 5000} → Score = 84.2404


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.438e+01, tolerance: 9.118e+00
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.644e+01, tolerance: 1.007e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.351e+01, tolerance: 1.2

[GridSearch] Params {'alpha': 100, 'max_iter': 10000} → Score = 83.7299

Best Parameters (Lasso): {'alpha': np.float64(100.0), 'max_iter': np.float64(1000.0)}
Best Score: 81.64238314268137


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.093e+04, tolerance: 5.646e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.785e+04, tolerance: 5.703e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.363e+04, tolerance: 5.7

,alpha,max_iter,score
0,0.001,1000,83.126021
1,0.001,5000,85.248647
2,0.001,10000,84.723131
3,0.010,1000,82.715455
4,0.010,5000,84.921600
5,0.010,10000,83.833754
6,0.100,1000,82.989844
7,0.100,5000,83.715798
8,0.100,10000,81.854600
9,1.000,1000,85.203892


In [17]:
y

0     353
1     366
2     329
3     407
4     406
5     447
6     464
7     485
8     529
9     603
10    552
11    489
12    439
13    359
14    379
15    417
16    412
17    438
18    337
19    267
20    185
21    217
22    224
23    235
24    235
25    211
26    215
27    233
28    247
29    243
30    221
31    154
32    163
33    240
34    207
35    236
36    202
37    241
38    235
39    211
40    339
41    245
42    227
43    283
44    297
Name: Homicide, dtype: int64

# train a lasso with alpha 0.001 and max iter 1000 and save it